In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Any, List, Optional, Union
import os
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.list import ListOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    ListOutputParser,
    JsonOutputParser
)
from langchain_core.pydantic_v1 import BaseModel, Field


class LLMProvider(ABC):
    """Abstract base class for LLM providers"""
    
    @abstractmethod
    def initialize_llm(self):
        """Initialize the LLM with appropriate credentials"""
        pass
    
    @property
    @abstractmethod
    def llm(self):
        """Return the initialized LLM instance"""
        pass


class GoogleLLMProvider(LLMProvider):
    """Concrete implementation for Google's Generative AI"""
    
    def __init__(self, model_name: str = "gemini-2.0-flash", api_key: Optional[str] = None):
        self._model_name = model_name
        self._api_key = api_key
        self._llm = None
        
    def initialize_llm(self):
        """Initialize the Google Generative AI model"""
        if not self._api_key:
            load_dotenv()
            self._api_key = os.getenv("GEMINI_API_KEY")
            
        if not self._api_key:
            raise ValueError("Google API key not found. Please provide it or set GEMINI_API_KEY in environment.")
            
        self._llm = ChatGoogleGenerativeAI(
            model=self._model_name, 
            google_api_key=self._api_key
        )
        
    @property
    def llm(self):
        """Return the initialized LLM instance"""
        if not self._llm:
            self.initialize_llm()
        return self._llm

In [ ]:


class OutputParser(ABC):
    """Abstract base class for output parsers"""
    
    @abstractmethod
    def get_parser(self):
        """Return the appropriate parser for the task"""
        pass


class CommaSeparatedParser(OutputParser):
    """Concrete implementation for comma-separated list parser"""
    
    def get_parser(self):
        return CommaSeparatedListOutputParser()


class JsonParser(OutputParser):
    """Concrete implementation for JSON output parser"""
    
    def __init__(self, model_class: BaseModel):
        self._model_class = model_class
        
    def get_parser(self):
        return JsonOutputParser(pydantic_object=self._model_class)
    
    def get_format_instructions(self):
        return self.get_parser().get_format_instructions()


class TemplateBuilder(ABC):
    """Abstract base class for template builders"""
    
    @abstractmethod
    def build_template(self) -> ChatPromptTemplate:
        """Build and return the prompt template"""
        pass

In [ ]:




class HtmlAnalysisTemplateBuilder(TemplateBuilder):
    """Template builder for HTML analysis prompts"""
    
    def build_template(self) -> ChatPromptTemplate:
        return ChatPromptTemplate.from_template("""
**Act as an expert HTML developer and content strategist.** Your role is to generate a precise and comprehensive list of key questions that a user should answer in order to fully customize the **text content** of an HTML template.  

### **Instructions:**  
1. **Analyze the provided HTML template:**  
   - Identify all text-based elements, including headings, paragraphs, buttons, navigation links, form placeholders, and any other textual content.  
   - Exclude non-text elements such as images, colors, fonts, and layout settings.  

2. **Generate key questions:**  
   - Focus only on **text-related customization** (e.g., wording, tone, branding consistency, and localization).  
   - Ensure the questions guide the user in making informed choices about the **tone, clarity, and structure** of the content.  

3. **Organize the questions logically:**  
   - Begin with general content structure (e.g., page headings and main text) and move to specific elements (e.g., button labels and form placeholders).  
   - Cover both **static text** (like headings and paragraphs) and **interactive text** (like form labels and call-to-action buttons).  

4. **Limit the output to exactly 10 questions.**  

### **HTML Template:**  
```html
{html_template}
```  

### **Output Format:**  
- Provide the questions in a **single-line, comma-separated format**, without any additional text.  
- Example output:  
  ```
  1. What should be the main heading of the page?, 2. What introductory text should appear in the hero section?, 3. What should the call-to-action button say?, ...etc
  ```

IMPORTANT: Do not use commas (,) inside your questions, as I am using CommaSeparatedListOutputParser. Instead, use a dash (-) to separate elements within a question.

**Take a deep breath and work on this problem step by step.**  
""")


class HtmlCustomizationTemplateBuilder(TemplateBuilder):
    """Template builder for HTML customization prompts"""
    
    def __init__(self, format_instructions: str):
        self._format_instructions = format_instructions
        
    def build_template(self) -> ChatPromptTemplate:
        return ChatPromptTemplate.from_template("""
**Act as an expert HTML developer and content strategist.**  
You have extensive experience in web development and content customization, specializing in dynamically adapting HTML templates based on user preferences.  

### **Objective:**  
Your task is to analyze a given HTML template and modify its **text content** based on the responses provided by the user. The goal is to ensure that the final HTML output accurately reflects the user's preferences while maintaining structural integrity and proper formatting.  

### **Process:**  
1. **Analyze User Inputs:**  
   - Carefully examine the provided **questions and user responses** to understand their content preferences.  
   - Identify key phrases, themes, and context to determine what changes need to be applied to the template.  

2. **Modify the HTML Template:**  
   - Update **only the text content** within the template while preserving the existing HTML structure, including elements, attributes, and styles.  
   - Ensure that text replacements maintain coherence and proper alignment with the template's purpose.  
   - If any user input is unclear or missing, generate a reasonable default response that aligns with the context.  

3. **Ensure Readability & Formatting:**  
   - Maintain proper text spacing, line breaks, and formatting conventions.  
   - If necessary, adjust the HTML markup to improve readability without altering functionality.  

4. **Output the Updated HTML Template:**  
   - Provide the final modified HTML code while keeping the structure intact.  
   - Ensure that all user preferences are accurately reflected in the text content.  

### **Inputs:**  
#### **HTML Template:**  
```html  
{html_template}  
```  

#### **Questions and User Responses:**  
{Q_A_list}  

### **Expected Output:**  
A fully updated HTML template with text content modified according to the user's answers, ensuring clarity, relevance, and consistency.  
Your html code should be start with body <body> tag and end with body tag </body>.
please answer in json like below:
```json
{{
  "html_template": "<body>Updated HTML code here...</body>"
}}
```

{format_instructions}

Take a deep breath and work on this problem step by step.  
""",
partial_variables={"format_instructions": self._format_instructions}
)





C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
class LangChainProcessor:
    """Main processor class that combines all components"""
    
    def __init__(self, llm_provider: LLMProvider):
        self._llm_provider = llm_provider
        self._html_content = None
        
    def load_html_from_file(self, file_path: str) -> str:
        """Load HTML content from a file"""
        try:
            with open(file_path, 'r') as f:
                self._html_content = f.read()
            return self._html_content
        except Exception as e:
            raise IOError(f"Failed to load HTML file: {e}")
    
    def set_html_content(self, html_content: str):
        """Set HTML content directly"""
        self._html_content = html_content
        
    def generate_questions(self, template_builder: TemplateBuilder, output_parser: OutputParser) -> List[str]:
        """Generate questions for HTML customization"""
        if not self._html_content:
            raise ValueError("HTML content is not set. Please load or set HTML content first.")
            
        prompt = template_builder.build_template()
        parser = output_parser.get_parser()
        
        chain = prompt | self._llm_provider.llm | parser
        
        response = chain.invoke({
            'html_template': self._html_content
        })
        
        return response
    
    def customize_html(self, 
                      template_builder: TemplateBuilder,
                      output_parser: OutputParser,
                      qa_responses: Dict[str, Any]) -> Dict[str, Any]:
        """Customize HTML based on question-answer pairs"""
        if not self._html_content:
            raise ValueError("HTML content is not set. Please load or set HTML content first.")
            
        prompt = template_builder.build_template()
        parser = output_parser.get_parser()
        
        chain = prompt | self._llm_provider.llm | parser
        
        response = chain.invoke({
            'html_template': self._html_content,
            'Q_A_list': qa_responses
        })
        
        return response


# Output model for HTML customization
class HtmlTemplate(BaseModel):
    html_template: str = Field(description="Html template with updated content after customization")



In [3]:
# Example usage
if __name__ == "__main__":
    # Initialize the LLM provider
    llm_provider = GoogleLLMProvider()
    
    # Create the processor
    processor = LangChainProcessor(llm_provider)
    
    # Load HTML content
    processor.load_html_from_file('../test_html.html')
    
    # Generate questions
    template_builder = HtmlAnalysisTemplateBuilder()
    parser = CommaSeparatedParser()
    
    questions = processor.generate_questions(template_builder, parser)
    print("Generated Questions:")
    for idx, question in enumerate(questions, 1):
        print(f"{idx}. {question}")
    
    # Example QA responses
    website_content = {
        "What is the desired brand name- logo text for the website header and footer?": "TechNova",
        "What wording should be used for the main heading (H1) on the hero section of the landing page?": "Revolutionizing Your Digital Experience",
        "What descriptive text should accompany the main heading in the hero section to explain the business offering?": "Empower your business with cutting-edge solutions designed to streamline operations and maximize efficiency.",
        "What call-to-action text should be used on the primary button in the hero section?": "Get Started Today",
        "What heading should be used for the 'Features' section to best describe the benefits offered?": "Why Choose TechNova?",
        "What wording should be used for each of the individual feature titles- for example- Smart Automation- Advanced Analytics- Enterprise Security?": [
            "Smart Automation",
            "Advanced Analytics",
            "Enterprise Security",
            "Seamless Integration"
        ],
        "What wording should be used for the 'Testimonials' section heading to convey customer satisfaction?": "What Our Clients Say",
        "What is the desired wording for the call-to-action heading and supporting text in the email collection section?": {
            "heading": "Stay Updated with Our Latest Innovations",
            "text": "Subscribe to our newsletter for exclusive insights, updates, and special offers."
        },
        "What placeholder text should be used in the email input field of the email collection form?": "Abdelrahman.m2922@gmail.com",
        "What wording should be used for the navigation links (e.g.- Features- Testimonials- Pricing- Contact)?": [
            "Features",
            "Testimonials",
            "Pricing",
            "Contact"
        ]
    }
    
    # Customize HTML with responses
    json_parser = JsonParser(HtmlTemplate)
    customization_builder = HtmlCustomizationTemplateBuilder(json_parser.get_format_instructions())
    
    customized_html = processor.customize_html(
        customization_builder,
        json_parser,
        website_content
    )
    
    print("\nCustomized HTML Template:")
    print(customized_html['html_template'])

Generated Questions:
1. 1. What is the desired brand name or logo text to replace "Quantum" in the header and footer?
2. 2. What text should replace "Modern Solution for Your Business" in the hero section's main headline- and what is the desired tone?
3. 3. What is the primary call-to-action for the hero section- and what text should be used for the "Start Free Trial" button?
4. 4. What names should be used for the navigation links- replacing "Features- Testimonials- Pricing- Contact"?
5. 5. What heading text should be used for the Features section- and what descriptive text should follow?
6. 6. What are the desired titles and descriptions for each feature card- replacing "Smart Automation"
7. Advanced Analytics
8. Enterprise Security and their respective descriptions?
9. 7. What heading text should be used for the Testimonials section- and what descriptive text should follow?
10. 8. What testimonial text- author name- and company name should be displayed- replacing the existing exampl